#### Author: Domitille Jarrige
#### Date: 2022-09-21

# Transposable elements mining in the _Dioszegia hungarica_ PDD-24b-2 genome


#### Module imports

In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import re

## Building TE library

Transposable elements (TE) were searched using protein homology to known TE from other fungi and two _de novo_ methods (RepeatModeler and EDTA).

### Protein homology method

BLAST+tblastx v2.11.0 was used with curated intern TE sequences (with their typical TE protein domains) from other fungi (_Saitozyma podzolica_, _Cryptococcus neoformans_, _Cryptococcus gattii_, _Rhodotorula toruloides_, _Candida glabrata_) as queries (exemple command line below).

`tblastx -query /path/to/transposon_queries/TE.fasta -subject /path/to/assembly/Diohu1_AssemblyScaffolds.fasta -outfmt 6 -out transposon_queries/TE_hits.txt`

The tblastx hits were then enlarged to include 6000 bp of their flanking sequences. This was done with Biopython in a jupyter notebook as in the following exemple:

##### Loading tblastx results table

In [2]:
transposons_df = pd.read_csv("transposon_queries/TE_hits.tsv", sep="\t", header=None, 
                             names=["transposon_name", "contig_name", "%identity","alignement_length","mismatches","gap_open", "t_start", 
                                    "t_end", "diohu_start", "diohu_end", "e_value", "score"])

The results tables have the following format:

In [3]:
transposons_df.head()

,transposon_name,contig_name,%identity,alignement_length,mismatches,gap_open,t_start,t_end,diohu_start,diohu_end,e_value,score
0,Mock_TE_1,contig_16,35.146,239,155,0,1636,2352,2302085,2301369,2.210000e-92,189.0
1,Mock_TE_1,contig_16,38.967,213,130,0,220,858,2303747,2303109,2.210000e-92,172.0
2,Mock_TE_1,contig_16,44.737,38,21,0,1129,1242,2302925,2302812,2.210000e-92,45.1
3,Mock_TE_1,contig_11,34.483,145,95,0,145,579,264963,265397,1.720000e-45,110.0
4,Mock_TE_1,contig_11,38.636,88,54,0,595,858,265422,265685,1.720000e-45,78.0


##### Extracting TE names

In [4]:
trans_list = list(transposons_df.groupby("transposon_name").sum().index)

In [5]:
print(trans_list)

['Mock_TE_1', 'Mock_TE_2', 'Mock_TE_3', 'Mock_TE_4', 'Mock_TE_5', 'Mock_TE_6']


##### Extracting putative TE sequences from _D. hungarica_ PDD-24b-2 genome 

In [10]:
for trans in trans_list:
    tmp_df = transposons_df[transposons_df["transposon_name"] == trans]
    
    # The highest scoring hit for each TE name is extracted
    contig = ", ".join(tmp_df.loc[(tmp_df["score"] == tmp_df["score"].max()), "contig_name"].values) 
    start = ", ".join(tmp_df.loc[(tmp_df["score"] == tmp_df["score"].max()), "diohu_start"].values.astype(str))
    end = ", ".join(tmp_df.loc[(tmp_df["score"] == tmp_df["score"].max()), "diohu_end"].values.astype(str))
    print(f"{trans}: best hit in {contig} at pos {start}, {end}")
    try:
        start_coord = int(start.split(", ")[0])
        end_coord = int(end.split(", ")[0])
    except ValueError:
        continue
    parser = SeqIO.parse("Diohu1_AssemblyScaffolds.fasta", format="fasta")
    for chromosome in parser:
        if chromosome.id == contig:
            record = SeqRecord(Seq(chromosome.seq[start_coord-6000:end_coord+6000]))
            record.id = contig + "putative " + trans
            record.description = "putative transposon region"
            
            # The files can be written but here we will just print their name
            file_name = contig + "_putative_" + trans + ".fasta"
            print(f"File: {file_name}")
            #SeqIO.write(record, file_name, format="fasta")

Mock_TE_1: best hit in contig_16 at pos 2302085, 2301369
File: contig_16_putative_Mock_TE_1.fasta
Mock_TE_2: best hit in contig_16 at pos 2302007, 2301321
File: contig_16_putative_Mock_TE_2.fasta
Mock_TE_3: best hit in contig_16 at pos 2302007, 2301369
File: contig_16_putative_Mock_TE_3.fasta
Mock_TE_4: best hit in contig_16 at pos 2302007, 2301369
File: contig_16_putative_Mock_TE_4.fasta
Mock_TE_5: best hit in contig_16 at pos 2302100, 2301369
File: contig_16_putative_Mock_TE_5.fasta
Mock_TE_6: best hit in contig_16 at pos 2302085, 2301369
File: contig_16_putative_Mock_TE_6.fasta


### RepeatModeler _de novo_ method

RepeatModeler [(Flynn et al. 2020)](https://doi.org/10.1073/pnas.1921046117) is available here: https://www.repeatmasker.org/RepeatModeler/

RepeatModeler v2.0.3 was used.

`BuildDatabase -name Dioszegia_hungarica Diohu1_AssemblyScaffolds.fasta`

`RepeatModeler -database Dioszegia_hungarica -LTRStruct >& repeat_modeler/repeat_modeler.log`

A multifasta library of putative TE was produced.

### EDTA _de novo_ method

The Extensive de novo TE Annotator (EDTA) pipeline [(Bell et al. 2022)](https://doi.org/10.1111/1755-0998.13489) is available here: https://github.com/oushujun/EDTA

`perl EDTA.pl --genome Diohu1_AssemblyScaffolds.fasta`

A multifasta library of putative TE was produced.

## Putative TE curation

Protein domains typical of TE (transposase, reverse transcriptase, RNase H, integrase, aspartic protease, gag domains) were then searched in the putative sequences from the three methods with CD Search ([Marchler-Bauer and Bryant 2004](https://doi.org/10.1093/nar/gkh454); [Marchler-Bauer et al. 2017](https://doi.org/10.1093/nar/gkw1129)). CD Search is accessible here: https://www.ncbi.nlm.nih.gov/Structure/cdd/wrpsb.cgi.

Terminal inverted repeats (TIR) and Target Site Duplications (TSD) were also searched to classify the TE according to [(Wicker et al. 2007)](https://doi.org/10.1038/nrg2165).

**For each TE family a unique candidate with TE domains was selected for the final library (see File S1)** 

## TE screening with RepeatMasker

Lastly the number and length of TE-related sequences (including full length and truncated copies) in the _Dioszegia hungarica_ PDD-24b-2 genome were calculated using RepeatMasker v4.1.2-p1.

`RepeatMasker -lib Supplementary_file_S1_for_RM.fasta Diohu1_AssemblyScaffolds.fasta -dir repeatmasker/ -pa 8`

The RepeatMasker outpule file was then parsed, to assemble the TE fragments, with the perl tool "One code to find them all" [(Bailly-Bechet et al. 2014)](https://doi.org/10.1186/1759-8753-5-13), available here: http://doua.prabi.fr/software/one-code-to-find-them-all

`./build_dictionary.pl -rm repeat_masker/Diohu1_AssemblyScaffolds.fasta.out --unknown > repeat_masker/one_code_to_find_them_all/ltr_dictionary`

`./one_code_to_find_them_all.pl --rm repeat_masker/Diohu1_AssemblyScaffolds.fasta.out --ltr repeat_masker/one_code_to_find_them_all/ltr_dictionary`

#### Table of TE

##### Stats of each TE family by contig

In [11]:
files = os.scandir("repeat_masker/")
paper_transposon_df = pd.DataFrame(columns=["Contig", "Family", "Element", "Length", "Fragments", "Copies", "Solo_LTR", "Total_Bp", "Cover"])

contig_name = r"(contig_[0-9]*)"

for f in files:
    if f.name.endswith(".copynumber.csv"):
        try :
            x = re.search(contig_name, f.name).group(1)
        except AttributeError:
            continue
        tmp_df = pd.read_csv(f, sep="\t", comment="#")
        tmp_df["Contig"] = x
        paper_transposon_df = pd.concat([paper_transposon_df, tmp_df])

# Beginning of TE table by contig
paper_transposon_df.head(20)

,Contig,Family,Element,Length,Fragments,Copies,Solo_LTR,Total_Bp,Cover
0,contig_11,LINE,TE-007_fam-06,2548,1,1,NaN,115,0.01
1,contig_11,LINE,TE-008_fam-07,2700,2,2,NaN,3192,0.29
2,contig_11,LTR/Copia,fam-11_I,5119,2,2,0.0,312,0.03
3,contig_11,LTR/Copia,fam-15_I,6917,11,8,2.0,8010,0.72
4,contig_11,LTR/Copia,fam-16_I,247,6,4,2.0,2284,0.21
5,contig_11,LTR/Gypsy,fam-09_I,5803,10,9,5.0,3042,0.27
0,contig_16,DNA/TIR/Mutator,TE-001_fam-02,3082,2,1,NaN,2647,0.11
1,contig_16,LINE,TE-008_fam-07,2700,3,3,NaN,3336,0.14
2,contig_16,LTR/Copia,fam-10_LTR,169,2,1,1.0,168,0.01
3,contig_16,LTR/Gypsy,fam-09_I,206,3,3,2.0,5798,0.25


##### Summarized TE families statistics

This table was the basis for **Table S1**. The annotation on the Class, Order, Superfamily, size, TIR, TSD and domains of the representative TE specimen from the final library (**File S1**) was added.

In [8]:
paper_transposon_df["Fragments"] = paper_transposon_df["Fragments"].astype('int64')
paper_transposon_df["Copies"] = paper_transposon_df["Copies"].astype('int64')
paper_transposon_df["Total_Bp"] = paper_transposon_df["Total_Bp"].astype('int64')
paper_transposon_df.groupby("Element").sum()[["Fragments", "Copies", "Solo_LTR", "Total_Bp"]]

,Fragments,Copies,Solo_LTR,Total_Bp
Element,,,,
TE-001_fam-02,10,7,0.0,10982
TE-002_fam-02,5,5,0.0,1895
TE-003_fam-03,10,10,0.0,7666
TE-004_fam-01,1,1,0.0,402
TE-005_fam-04,1,1,0.0,395
TE-006_fam-05,19,18,0.0,12979
TE-007_fam-06,3,2,0.0,2876
TE-008_fam-07,14,13,0.0,12924
fam-08_I,9,6,2.0,4408


#### Session information and modules

In [9]:
%load_ext watermark
%watermark -t -d -v -m
print("")
%watermark -w -p pandas,Bio,jupyterlab

Python implementation: CPython
Python version       : 3.9.0
IPython version      : 8.0.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 3, GenuineIntel
CPU cores   : 12
Architecture: 64bit


pandas    : 1.3.3
Bio       : 1.79
jupyterlab: 3.3.4

Watermark: 2.3.1

